# Fase 01 - Obtenció de dades

In [8]:
from downloadDiaryOfSessions.main import download
from googleSync.main import sync_folder

import json
from os.path import basename, isdir, isfile
from os import listdir, makedirs
import pandas as pd
from pandas import json_normalize
from pdf2image import convert_from_path
import requests
import subprocess

## 1.- Descarregar diaris de sessions

In [9]:
sync_folder('.data', 'TFG-DATA')

usage: ipykernel_launcher.py [--auth_host_name AUTH_HOST_NAME]
                             [--noauth_local_webserver]
                             [--auth_host_port [AUTH_HOST_PORT ...]]
                             [--logging_level {DEBUG,INFO,WARNING,ERROR,CRITICAL}]
ipykernel_launcher.py: error: unrecognized arguments: -f C:\Users\jjescandell\AppData\Roaming\jupyter\runtime\kernel-c7d79c7d-02c9-400e-8778-6799763f7be5.json


SystemExit: 2

C:\Users\jjescandell\Anaconda3\envs\tfg-22-env\lib\site-packages\IPython\core\interactiveshell.py:3406: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [3]:
legislatures = [11, 12, 13, 14]
max_errors = 50
dscd_path = './.data/diary_session_congress_deputies_pdfs'
dss_path = './.data/diary_session_senate_pdfs'

def diary_session_congress_deputies_url(legislature, plenary):
    return f"https://www.congreso.es/public_oficiales/L{legislature}/CONG/DS/PL/DSCD-{legislature}-PL-{plenary}.PDF"

def diary_session_senate_url(legislature, plenary):
    return f"https://www.senado.es/legis{legislature}/publicaciones/pdf/senado/ds/DS_C_{legislature}_{plenary}.PDF"

### 1.1.- Diaris de sessions del congress de diputats

In [3]:
if not isdir(dscd_path):
    makedirs(dscd_path)

for current_legislature in legislatures:
    current_plenary = 1
    consecutive_errors = 0
    while consecutive_errors < max_errors:
        url = diary_session_congress_deputies_url(current_legislature, current_plenary)
        file = f"dscd-{current_legislature}-{current_plenary:03d}"
        if download(url, dscd_path, file):
            consecutive_errors = 0
        else:
            consecutive_errors += 1

        current_plenary += 1

### 1.2.- Diaris de sessions del senat

In [4]:
if not isdir(dss_path):
    makedirs(dss_path)

for current_legislature in legislatures:
    current_plenary = 1
    consecutive_errors = 0
    while consecutive_errors < max_errors:
        url = diary_session_senate_url(current_legislature, current_plenary)
        file = f"dss-{current_legislature}-{current_plenary:03d}"
        if download(url, dss_path, file):
            consecutive_errors = 0
        else:
            consecutive_errors += 1

        current_plenary += 1

## 2.- Convertir pagines de pdf a imatges

In [4]:
left_margin = 175
top_header_margin = 790
top_regular_margin = 296
right_margin = 1480
bottom_margin = 2160

dscd_files = [f'{dscd_path}/{file}' for file in listdir(dscd_path)]
dss_files = [f'{dss_path}/{file}' for file in listdir(dss_path)]
paths = dscd_files + dss_files

for path in paths:
    folder = f'./.data/images/{basename(path)[:-4]}'
    if not isdir(folder):
        makedirs(folder)
    else:
        continue
    pages = convert_from_path(path)
    page_index = 0
    for page in pages:
        top_margin = top_header_margin if page_index == 0 else top_regular_margin
        page\
            .crop((left_margin, top_margin, right_margin, bottom_margin))\
            .save(f'{folder}/{basename(path)[:-4]}-{page_index:03d}.jpeg', 'JPEG')
        page_index += 1

PDFInfoNotInstalledError: Unable to get page count. Is poppler installed and in PATH?

## 3.- Convertir imatges a text

In [ ]:
cmd_docker_build = 'docker build -t tfg-ocr $(pwd)/tesseractOcr'
cmd_docker_run = 'docker run -v $(pwd)/.data/images:/home/images -v $(pwd)/.data/texts:/home/texts tfg-ocr'

subprocess.call(cmd_docker_build, shell=True)
subprocess.call(cmd_docker_run, shell=True)

In [ ]:
all_texts_path = './.data/texts'

text_paths = listdir(all_texts_path)
for text_path in text_paths:
    if isfile(f'{all_texts_path}/{text_path}'):
        continue
    full_text_path = f'{all_texts_path}/full_{text_path}.txt'
    if isfile(full_text_path):
        continue

    text_pages_paths = sorted(listdir(f'{all_texts_path}/{text_path}'))
    with open(full_text_path, 'w') as full_text_file:
        for text_page_path in text_pages_paths:
            with open(f'{all_texts_path}/{text_path}/{text_page_path}') as page_text_file:
                full_text_file.write(page_text_file.read())

## 4.- Obtenir diputats

In [ ]:
def get_deputies(legislature):
    url_deputies = "https://www.congreso.es/busqueda-de-diputados?p_p_id=diputadomodule&p_p_lifecycle=2&p_p_state=normal&p_p_mode=view&p_p_resource_id=searchDiputados&p_p_cacheability=cacheLevelPage"

    payload={'_diputadomodule_idLegislatura': legislature,
             '_diputadomodule_genero': '0',
             '_diputadomodule_grupo': 'all',
             '_diputadomodule_tipo': '2',
             '_diputadomodule_nombre': '',
             '_diputadomodule_apellidos': '',
             '_diputadomodule_formacion': 'all',
             '_diputadomodule_filtroProvincias': '[]',
             '_diputadomodule_nombreCircunscripcion': ''}

    response = requests.request("POST", url_deputies, data=payload)
    return response.text

In [ ]:
pkl_path = './.data/deputies.pkl'
if isfile(pkl_path):
    print('from file')
    dp = pd.read_pickle(pkl_path)
else:
    print('download')
    df = None
    for current_legislature in legislatures:
        data = get_deputies(current_legislature)
        data_json = json.loads(data)
        if df is None:
            df = json_normalize(data_json['data'])
        else:
            new_df = json_normalize(data_json['data'])
            for index, row in new_df.iterrows():
                current_row_df = df[(df.apellidosNombre == row['apellidosNombre']) & (df.formacion == row['formacion'])]
                if current_row_df.empty:
                    df = pd.concat([df, row])
                else:
                    df.loc[(df.apellidosNombre == row['apellidosNombre']) & (df.formacion == row['formacion']), 'fchBaja'] = row['fchBaja']
                    df.loc[(df.apellidosNombre == row['apellidosNombre']) & (df.formacion == row['formacion']), 'idLegislatura'] = f'{current_row_df.iloc[0].idLegislatura},{row["idLegislatura"]}'

    df.to_pickle('./.data/deputies.pkl')